# Sales Analysis

*__Disclaimer__*: The datasets used in this notebook were taken from the tutorial provided by @KeithGalli (check out his GitHub). I followed his tutorial in YouTube, however, I tried to use other functions and methods I considered more suitable for the questions at hand.

Even though I believe this notebook is very self explanatory (and heavily commented), I'd recommend checking his tutorial on YouTube so you get an idea of the problem at hand.

The idea of this tutorial is to use __pandas__ to make analysis on a dataset that comprises the yearly sales of a business. We will import the data into python and navigate our way through the data to come up with answers to typical questions a business owner might be interested about regarding this data.

## Import necessary libraries

Let's start by importing libraries we will be using in this tutorial

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import os
import pandas as pd

## Merge all the CSV files into a single DataFrame

Now let's import the data, which comprises 12 months of sales. Then, let's combine all the data into a single DataFrame.

In [ ]:
# Merge all the files using Unix style pathname pattern
joined_files = os.path.join("./csv", "*.csv")

In [ ]:
# 'glob' is a Unix style pathname pattern expansion
# The next line will return a list with the paths to
# all the CSV files in the folder ./csv/
# Furthermore, the list will be in order (by month: Jan - Feb - etc.)
joined_list = glob.glob(joined_files)

In [ ]:
# Finally, concatenate all the CSV files into a single DataFrame
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
df.head()

# Save final DataFrame to CSV (not required, but good to know how to do it!)
# df.to_csv("all_data.csv", index=False)

## Clean up the data

The dataset has some rows with irrelevant data and missing information, so let's delete those entries so we have a clean dataset to work with.

In [ ]:
# Check for rows with NaN values
df[df["Order Date"].isna()]

In [ ]:
# Drop rows with NaN values
df = df.dropna(how="all")

In [ ]:
# There are some rows that repeat the header multiple times
df.loc[df["Order Date"] == "Order Date"]

In [ ]:
# Let's delete those lines. I will use the 'Order Date' to do so. You can use any
# column you find convenient
df = df.drop(df[df["Order Date"] == "Order Date"].index) 
# df = df[df["Order Date"] != "Order Date"]    # This is another way to achieve the same result

In [ ]:
# Let's index the DataFrame with the 'Order Date' -> Convert to 'datetime'
# object so we can manipulate dates easily
df["Order Date"] = pd.to_datetime(df["Order Date"])
df = df.set_index("Order Date")

In [ ]:
# Let's change the type of numeric columns from 'str' to 'int' and 'float'
df["Quantity Ordered"] = df["Quantity Ordered"].astype('int32')
# df["Order ID"] = df["Order ID"].astype('int32')
df["Price Each"] = df["Price Each"].astype('float')

## Analysis of the data

#### __Question 1__ What was the best month for sales? How much was earned that month?

To answer this question, let's multiply the price of each article by the quantity ordered, and store the value in a new column. Then we can add the results per month, and in this way we can figure out which was the best month for sales, as well as the amount earned on that month

In [ ]:
# Create new column 'Sales' = 'Quantity' * 'Price'
df["Sales"] = df["Quantity Ordered"] * df["Price Each"]

# Group sales per month
total_sales_monthly = df.groupby(by=[df.index.month])["Sales"].sum()

# Let's format our output so it looks nice
total_sales_monthly.map('{:,.2f}'.format)

From the table we can see that the best month was December, with total sales of \\$4,613,443.34, not too shabby!!! On ther hand, the worst month was January with only \\$1,822,256.73

Most of the time we will find that plots are a great way to convey results, so let's do that with our monthly sales

In [ ]:
# Let's plot a bar chart

# You can use the Series.plot() method from pandas, but I think is not as customizable as Matplotlib
# total_sales_monthly.plot(kind='bar', xlabel="Month", ylabel="Sales ($)", title="Monthly Sales")

# I will use Matplotlib
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
height = np.random.randint(100000, 500000, 10)
plt.bar(months, total_sales_monthly)

fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)

plt.xlabel("Month")
plt.ylabel("Sales ($)")
plt.title("Monthly Sales")
plt.xticks(rotation=30)

plt.show()

From the first question we can see that sales tend to peak by the end of the year. This is understandable, since the holiday season is around the corner. We know that dates such as Black Friday and Christmas contribute enourmously to the increase in sales compared to the rest of the year.

There is also some increase in sales during April: people are getting ready for summer break and businesses take advantage to finish stock by launching last minute deals. The rest of the year is relatively quiet.

#### __Question 2__: What city sold the most products?

This is similar to the previous question, but this time we need to group the data by city, and then add the total sales to figure out the city with the most sales throughout the year

In [ ]:
# Let's collect the city and state from the 'Purchase Address' column
# We can use Regex to very easily achieve this objective
# Notice we collect also the state since there might be cities with the same
# name but in different states
df["City"] = df["Purchase Address"].str.extract(r"[.+,](.*, [A-Z][A-Z])[ , .*]")

In [ ]:
# Group sales per city
# You can sort the values by adding the 'sort_values()' method
sales_per_city = df.groupby(["City"])["Sales"].sum() # .sort_values(ascending=False)

# Let's format our output so it looks nice
sales_per_city.map('{:,.2f}'.format)

Let's plot the results

In [ ]:
# I will use Matplotlib
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
height = np.random.randint(100000, 500000, 10)
plt.bar(sales_per_city.index, sales_per_city)

fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)

plt.xlabel("City")
plt.ylabel("Sales ($)")
plt.title("Yearly Sales per city")
plt.xticks(rotation=90, size=8)

plt.show()

It can be seen that San Francisco is the city with the most sales throughout the year. The dataset comprises sales of electronics and gadgets, and given the proximity of San Francisco to Silicon Valley, this makes sense.

Also it is worth pointing out that the cities with the most sales are also cities where people earn higher salaries, so consumers in cities like San Francisco, LA, NY, etc., have some buying power that allows them to spend more easily on these devices.

On the other hand, people living in cities such as Portland (ME) and Portland (OR), where salaries are lower, might not be able to afford electronic gadgets with the same frequency as people living in bigger cities.

#### __Question 3__: What time should we display advertisements to maximize likelihood of customer's buying a product?

In [ ]:
# Group sales per hours. In this case let's count the number of times
#  sale was made made
count_sales_hourly = df.groupby(by=[df.index.hour])["Quantity Ordered"].count()

# Let's format our output so it looks nice
count_sales_hourly.map('{:,}'.format)

Time to plot the results and get some insights

In [ ]:
# I will use Matplotlib
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
height = np.random.randint(100000, 500000, 10)
plt.plot(count_sales_hourly.index, count_sales_hourly)

fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)

plt.xlabel("Hour of day")
plt.ylabel("Number of orders")
plt.title("Hourly number of orders throughout the year")
plt.xticks(np.arange(0, 24, step=2))
plt.grid(True, linestyle=":")

plt.show()

We can see that between 11:00 - 13:00 and 18:00 - 20:00 are the times with maximum demand for products. Therefore, we might deploy an advertisement to run just before 11:00 or noon, and also just before 18:00 or 19:00 in order to increase our chances of people buying the products.

#### __Question 4__: What product are most often sold together? 

First, notice we have a column named 'Order ID'. If two or more products have the same order ID, then it means they were bought together. We can use this information to find which products are regularly bought together

In [ ]:
# Let's find the duplicated 'Order ID' in our DataFrame
df_duplicate_id = df[df["Order ID"].duplicated(keep=False)]
df_duplicate_id

In [ ]:
# Now let's combine the products with the same Order ID in a single column
# 'reset_index()' is required so the result is a DataFrame
df_order_id = df_duplicate_id.groupby("Order ID")["Product"].unique().str.join(',').reset_index()
df_order_id

In [ ]:
# Now let's import some libraries we will use to iterate and find most common pairs of products
# that are bought together
from collections import Counter
from itertools import combinations

In [ ]:
# Start the counter
count = Counter()

# Iterate over each row in 'Products'
for row in df_order_id["Product"]:
    # Split row at the comma
    row_list = row.split(',')
    
    # Update counter of possible combinations of products (only pairs)
    # If you want to check the most common 'trio' bought together, just change
    # the number 2 for a 3. Similarly, if you'd like to know the most popular
    # 4 items that were bought together, just update the number to 4, and so on...
    count.update(Counter(combinations(row_list, 2)))

# Print 10 most common pair of objects bought together
count.most_common(10)

Even though the result on the screen might look messy, it is actually easy to read. Just check the beginning of the output and you will see that the list is sorted from the most common pair of elements that were bought together, to the least popular pair. So we can conclude that the pair comprised of an *iPhone* with a *Lightning Charging Cable* is the most popular pair of gadgets that were bought together (a whopping 1,002 times).

This information is useful to businesses because they can create promotions based on the buying habits of most of their customers; or in this case, if a customer is buying an iPhone, the store might use the information we just found and offer the customer a Lightning Charging Cable. This is a strategy that might increase sales and help businesses with their stock turnover.

#### __Question 5__: What product sold the most? Why?

In [ ]:
# Let's group the DataFrame by 'Product', and see how many units were sold throughout the year
most_popular_product = df.groupby("Product")["Quantity Ordered"].sum() # .sort_values(ascending=False)

# Let's format our output so it looks nice
most_popular_product.map('{:,}'.format)

In [ ]:
# Let's compute average price of prodcts throughout the year
prices = df.groupby("Product")["Price Each"].mean() # .sort_values(ascending=False)

# Let's format our output so it looks nice
prices.map('{:,}'.format)

Let's plot the results so we have a visual aid to interpret this results.

In [ ]:
# I will use Matplotlib
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
height = np.random.randint(100000, 500000, 10)
plt.bar(most_popular_product.index, most_popular_product, color="green")

fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)

plt.xlabel("Product")
plt.ylabel("Units sold", color="green")
plt.title("Yearly units sold")
plt.xticks(rotation=90, size=8)
plt.yticks(color="green")

# Let's plot prices on top
ax2 = ax.twinx()
ax2.set_ylabel('Price ($)', color="red")
ax2.plot(most_popular_product.index, prices, color="red")
ax2.tick_params(axis='y', colors='red')

fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax2.yaxis.set_major_formatter(tick)


plt.show()

Incredibly, the most popular item were the AAA Batteries (4-pack). But come to think of it, it makes sense, since a lot of gadgets still require this type of batteries (like a TV remote control, some toys, or the control for your PS5/Xbox, etc.), and they need to be replaced regularly. 